In [3]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook, curdoc
from bokeh.palettes import Category20
output_notebook()
curdoc().theme = "dark_minimal"
from tqdm.notebook import tqdm, trange

from py_vollib.black_scholes_merton import black_scholes_merton
from py_vollib.black_scholes_merton.greeks.analytical import delta, gamma, vega, theta, rho
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility

Loading BokehJS ...

In [4]:
df = pd.read_csv("../data/case2/training_pricepaths.csv", index_col=0)
df

,underlying,call65,call70,call75,call80,call85,call90,call95,call100,call105,...,put90,put95,put100,put105,put110,put115,put120,put125,put130,put135
0,100.000000,35.436537,30.436449,25.465460,20.544501,15.718957,11.091298,6.883928,3.493461,1.333979,...,1.091298,1.883928,3.493461,6.333979,10.375479,15.090290,20.025352,25.011182,30.008593,35.010526
1,99.703351,35.117638,30.116325,25.143759,20.221087,15.395046,10.773197,6.591873,3.268877,1.212482,...,1.069846,1.888522,3.565526,6.509131,10.632709,15.380026,20.322307,25.309459,30.307651,35.311125
2,99.781739,35.246585,30.252950,25.291467,20.385660,15.585513,11.000335,6.856841,3.536347,1.409663,...,1.218595,2.075101,3.754608,6.627924,10.649909,15.334966,20.254847,25.235405,30.231430,35.233703
3,99.989922,35.412877,30.423416,25.464844,20.560375,15.759372,11.169538,7.013417,3.664809,1.492569,...,1.179616,2.023495,3.674888,6.502648,10.479730,15.139596,20.050467,25.028339,30.023364,35.024824
4,99.348765,34.765348,29.783212,24.834189,19.944124,15.167839,10.623611,6.550014,3.329964,1.305116,...,1.274847,2.201250,3.981199,6.956351,11.042635,15.753561,20.681516,25.664342,30.660513,35.661443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,100.032705,35.043239,30.047080,25.055534,20.076258,15.133960,10.312569,5.876522,2.432486,0.601069,...,0.279864,0.843816,2.399781,5.568363,10.044239,14.972941,19.967656,24.967330,29.967303,34.967299
597,100.878248,35.889777,30.893417,25.901330,20.920246,15.971332,11.125318,6.606533,2.957641,0.838377,...,0.247069,0.728284,2.079392,4.960129,9.253432,14.134146,19.122762,24.121870,29.121781,34.121768
598,100.644919,35.659095,30.663181,25.672049,20.693025,15.748823,10.914257,6.423055,2.829666,0.781541,...,0.269338,0.778136,2.184747,5.136623,9.473957,14.366037,19.355988,24.355194,29.355112,34.355101
599,101.390634,36.407339,31.410827,26.418535,21.436438,16.482656,11.616822,7.032380,3.233859,0.935217,...,0.226189,0.641746,1.843225,4.544583,8.753137,13.621763,18.610260,23.609462,28.609390,33.609381


In [5]:
p = figure(height=200, width=900)
p.line(df.index, df.underlying)
show(p)

In [6]:
p = figure(height=200, width=900)
palette = Category20[20]
cols = ["call90", "call95", "call100", "call105", "call110"]
for i, col in enumerate(cols):
    p.line(df.index, df[col], line_color=palette[i%20], legend_label=col)
show(p)

In [7]:
p = figure(height=200, width=900)
palette = Category20[20]
cols = ["put90", "put95", "put100", "put105", "put110"]
for i, col in enumerate(cols):
    p.line(df.index, df[col], line_color=palette[i%20], legend_label=col)
show(p)

In [8]:
1/12

0.08333333333333333

In [9]:
len(df)

601

In [10]:
iv = {}
for col in df.columns:
    flag = col[0]
    if flag == 'u':
        continue
    strike = int(col[3:]) if flag == 'p' else int(col[4:])
    l = [implied_volatility(df[col][i], df.underlying[i], strike, 0.25-i/3600, 0, 0, flag) for i in range(len(df))]
    iv[col] = l
iv = pd.DataFrame(iv)
iv

,call65,call70,call75,call80,call85,call90,call95,call100,call105,call110,...,put90,put95,put100,put105,put110,put115,put120,put125,put130,put135
0,0.523355,0.446913,0.380012,0.322176,0.273039,0.232310,0.199757,0.175192,0.158460,0.149432,...,0.232310,0.199757,0.175192,0.158460,0.149432,0.148000,0.154072,0.167568,0.188423,0.216577
1,0.515440,0.439233,0.372665,0.315283,0.266737,0.226750,0.195102,0.171613,0.156133,0.148541,...,0.226750,0.195102,0.171613,0.156133,0.148541,0.148731,0.156618,0.172124,0.195185,0.225745
2,0.528684,0.452670,0.386174,0.328725,0.279960,0.239593,0.207392,0.183172,0.166779,0.158085,...,0.239593,0.207392,0.183172,0.166779,0.158085,0.156982,0.163379,0.177198,0.198373,0.226844
3,0.520800,0.446925,0.382272,0.326382,0.278899,0.239543,0.208091,0.184361,0.168201,0.159489,...,0.239543,0.208091,0.184361,0.168201,0.159489,0.158119,0.164003,0.177065,0.197238,0.224468
4,0.513607,0.441283,0.377984,0.323251,0.276728,0.238134,0.207243,0.183871,0.167868,0.159107,...,0.238134,0.207243,0.183871,0.167868,0.159107,0.157483,0.162905,0.175297,0.194592,0.220732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.535585,0.464772,0.402626,0.348669,0.302531,0.263923,0.232612,0.208408,0.191157,0.180728,...,0.263923,0.232612,0.208408,0.191157,0.180728,0.177014,0.179923,0.189375,0.205302,0.227647
597,0.550550,0.477839,0.414014,0.358593,0.311202,0.271550,0.239404,0.214574,0.196906,0.186270,...,0.271550,0.239404,0.214574,0.196906,0.186270,0.182559,0.185681,0.195558,0.212122,0.235316
598,0.560427,0.485549,0.419852,0.362842,0.314141,0.273448,0.240525,0.215178,0.197249,0.186604,...,0.273448,0.240525,0.215178,0.197249,0.186604,0.183133,0.186743,0.197352,0.214892,0.239303
599,0.579381,0.500437,0.431084,0.370791,0.319150,0.275842,0.240611,0.213251,0.193593,0.181497,...,0.275842,0.240611,0.213251,0.193593,0.181497,0.176844,0.179537,0.189488,0.206627,0.230889


In [11]:
def plot_volatility_curve(iv, t, side):
    p = figure(height=200)
    if side == "c":
        x = iv.iloc[t, :15]
        idx = [int(i[4:]) for i in x.index]
    elif side == "p":
        x = iv.iloc[t, 15:]
        idx = [int(i[3:]) for i in x.index]
    p.line(idx, x)
    show(p)
    return x.to_numpy()

In [28]:
x = plot_volatility_curve(iv, 0, 'c')

In [29]:
y = plot_volatility_curve(iv, 283, 'c') # underlying = 95.2

In [30]:
z = plot_volatility_curve(iv, 478, 'c') # underlying = 99.8

In [31]:
p = figure(height=200)
p.line(np.arange(65, 140, 5), x - x[7])
p.line(np.arange(65, 140, 5) + 5, y - y[6], line_color="red")
p.line(np.arange(65, 140, 5), z - z[7], line_color="yellow")
show(p)

In [20]:
for i, v in zip(np.arange(65, 140, 5), x):
    print(f'"{i}": {0.},')

"65": 0.0,
"70": 0.0,
"75": 0.0,
"80": 0.0,
"85": 0.0,
"90": 0.0,
"95": 0.0,
"100": 0.0,
"105": 0.0,
"110": 0.0,
"115": 0.0,
"120": 0.0,
"125": 0.0,
"130": 0.0,
"135": 0.0,


In [17]:
p = figure(height=400, width=900)
palette = Category20[20]
cols = ["put85", "put90", "put95", "put100", "put105", "put110", "put115"]
for i, col in enumerate(cols):
    p.line(iv.index, iv[col], line_color=palette[i%20], legend_label=col)
show(p)

In [12]:
iv.iloc[10, :15]

call65     0.516334
call70     0.445048
call75     0.382450
call80     0.328056
call85     0.281490
call90     0.242458
call95     0.210725
call100    0.186100
call105    0.168425
call110    0.157569
call115    0.153424
call120    0.155896
call125    0.164906
call130    0.180385
call135    0.202275
Name: 10, dtype: float64

# Orderbook
- NBBO is always 0.72222 = 13/18
- looks like NBBO is constructed as true price +- 13/36
- all bid < 0.1 are cut off (no liquidity)

In [3]:
a = np.load("../data/case2/pricespaths.pickle", allow_pickle=True)
a['0'].keys()

dict_keys(['SPY', 'SPY65C', 'SPY65P', 'SPY70C', 'SPY70P', 'SPY75C', 'SPY75P', 'SPY80C', 'SPY80P', 'SPY85C', 'SPY85P', 'SPY90C', 'SPY90P', 'SPY95C', 'SPY95P', 'SPY100C', 'SPY100P', 'SPY105C', 'SPY105P', 'SPY110C', 'SPY110P', 'SPY115C', 'SPY115P', 'SPY120C', 'SPY120P', 'SPY125C', 'SPY125P', 'SPY130C', 'SPY130P', 'SPY135C', 'SPY135P'])

In [14]:
a['98']['SPY']['bid']

[[100.32751033112092, 5],
 [100.31751033112093, 10],
 [100.30751033112092, 17],
 [100.29751033112092, 31],
 [100.28751033112093, 56],
 [100.27751033112092, 100],
 [100.26751033112093, 177],
 [100.25751033112093, 316],
 [100.24751033112092, 562]]

In [32]:
bid, ask = [], []
mid = []
sym = 'SPY110C'
for i in a.values():
    bid.append(i[sym]['bid'][0][0] if i[sym]['bid'] else np.nan)
    ask.append(i[sym]['ask'][0][0] if i[sym]['ask'] else np.nan)
    mid.append((i[sym]['bid'][0][0] + i[sym]['ask'][0][0]) / 2 if i[sym]['bid'] and i[sym]['ask'] else np.nan)
p = figure(height=300, width=800)
p.line(np.arange(len(mid)), mid)
p.line(np.arange(len(bid)), bid, line_color="red")
p.line(np.arange(len(ask)), ask, line_color="red")
show(p)

In [26]:
np.array(bid) - np.array(ask)

array([-0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72222222,
       -0.72222222, -0.72222222, -0.72222222, -0.72222222, -0.72

In [27]:
from py_vollib.black_scholes_merton import black_scholes_merton
from py_vollib.black_scholes_merton.greeks.analytical import delta, gamma, vega, theta, rho
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility
S = 49
K = 50
r = .05
t = 0.3846
q = 0
sigma = 0.2
flag = 'c'
delta_calc = gamma(flag, S, K, t, r, sigma, q)
# 0.521601633972
delta_text_book = 0.522
delta_calc

0.06554537725247869